<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#EDA" data-toc-modified-id="EDA-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>EDA</a></span><ul class="toc-item"><li><span><a href="#Dataset-Information" data-toc-modified-id="Dataset-Information-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Dataset Information</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Data-Visualizations" data-toc-modified-id="Data-Visualizations-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Data Visualizations</a></span><ul class="toc-item"><li><span><a href="#Target-Variable" data-toc-modified-id="Target-Variable-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Target Variable</a></span></li><li><span><a href="#Features" data-toc-modified-id="Features-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Features</a></span></li></ul></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Active-Model" data-toc-modified-id="Active-Model-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Active Model</a></span></li><li><span><a href="#Indoors" data-toc-modified-id="Indoors-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Indoors</a></span></li><li><span><a href="#With-Friends" data-toc-modified-id="With-Friends-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>With Friends</a></span></li></ul></li><li><span><a href="#Evaluate-Best-Models:-RFC" data-toc-modified-id="Evaluate-Best-Models:-RFC-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluate Best Models: RFC</a></span></li></ul></div>

# Introduction

First: Go out and find a dataset of interest. It could be from one of our recommended resources, some other aggregation, or scraped yourself. Just make sure it has lots of variables in it, including an outcome of interest to you.

Second: Explore the data. Get to know the data. Spend a lot of time going over its quirks and peccadilloes. You should understand how it was gathered, what's in it, and what the variables look like.

Third: Model your outcome of interest. You should try several different approaches and really work to tune a variety of models before using the model evaluation techniques to choose what you consider to be the best performer. Make sure to think about explanatory versus predictive power and experiment with both.

So, here is the deliverable: Prepare a slide deck and 15 minute presentation that guides viewers through your model. Be sure to cover a few specific things:

A specified research question your model addresses
How you chose your model specification and what alternatives you compared it to
The practical uses of your model for an audience of interest
Any weak points or shortcomings of your model

# Imports

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from matplotlib.legend_handler import HandlerLine2D


from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import ensemble
from sklearn.neural_network import MLPClassifier
from scipy.stats import ttest_ind

import operator
import os


%matplotlib inline

# Options for pandas
pd.options.display.max_columns = 200
pd.options.display.max_rows = 500

In [8]:
#use sample user to create separate list of features and target labels to be used for cell below

# sample_user = pd.read_csv('data/1538C99F-BA1E-4EFB-A949-6C7C47701B20.features_labels.csv.gz')

# list of one-hot encoded labels
#label_col = label_col = [col for col in list(sample_user.columns) if col[:6] == 'label:']
# list of explanatory features
#all_feature_list = [col for col in list(sample_user.columns) if col not in label_col and col != 'timestamp']

In [9]:
# #create master dataframe with all users
# directory = os.fsencode('data/')
# master_df = pd.DataFrame()

# for i,file in enumerate(os.listdir(directory)):
#     filename = os.fsdecode(file)
#     if filename.endswith(".gz") or filename.endswith(".csv"): 
#         user = pd.read_csv('data/' + filename)
#         #labels each observation by user number
#         user['user_num'] = i
        
#         #linear interpolate explanatory columns for each user df individually
#         nan_col = [col for col in all_feature_list if user[col].isnull().sum()/user[col].isnull().count()*100 > 0]
        
#         for col in nan_col:
#             user[col].interpolate(method='linear', limit_direction='both', inplace=True)
        
#         #fill nan with zero for dummy labels
#         for col in label_col:
#             user[col].fillna(0, axis=0, inplace=True)
#         #add user dataframe to master dataframe
#         master_df = master_df.append(user)
# master_df.to_csv('data/master_df.csv')

In [10]:
master_df = pd.read_csv('data/master_df.csv')

# EDA

## Dataset Information

The are 60 participants within this [kaggle dataset](https://www.kaggle.com/yvaizman/the-extrasensory-dataset#user2.features_labels.csv) (34 females and 26 males). Majority of participants are UCSD undergraduate and graduate students or research assistants. The average number of days users participated was 7.6 days (std: 3.2). 

The data from each user was collected by a mobile app called ExtraSensory. The majority of the data was collected by built-in sensors within the user's phone and provided smart watches. The sensor data was collected in a 20 second window with intervals of one minute and some time gaps in between. The sensors range from movement sensors such as accelorometer, gyrometer and magnetometer to audio and location sensors. Within the dataset there are different metrics calculated in relation to each sensor such as: mean, median, std, etc. There are missing observations within some sensor columns such as location and audio because of user's personal privacy reasons. Additional binary columns were self-reported by the user such as wifi available and time of day. 
 
The user also self-reported what activity they were doing or what location they were at within the recorded timeframe. The user was able to choose from general movement activities (e.g. lying down, sitting, standing in place, walking, or running), specific movement acitivies (e.g. eating, texting or exercising) or general location (e.g. gym, workplace, or home). 

Sensor Information according to [ExtraSensory UCSD dataset website](http://extrasensory.ucsd.edu/): 

- accelerometer: Tri-axial direction and magnitude of acceleration. 40Hz 
- gyroscope:	Rate of rotation around phone's 3 axes. 40Hz
- magnetometer:	Tri-axial direction and magnitude of magnetic field. 40Hz
- watch accelerometer:	Tri-axial acceleration from the watch. 25Hz
- watch compass:	Watch heading (degrees). nC samples (whenever changes in 1deg)
- location:	Latitude, longitude, altitude, speed, accuracies. nL samples (whenever changed enough)
- location (quick):	Quick location-variability features (no absolute coordinates) calculated on the phone.
- audio:	22kHz for ~20sec. Then 13 MFCC features from half overlapping 96msec frames.
- audio magnitude:	Max absolute value of recorded audio, before it was normalized.	
- phone state:	App status, battery state, WiFi availability, on the phone, time-of-day.
- additional:	Light, air pressure, humidity, temperature, proximity. If available sampled once in session

In [11]:
master_df.head()

,Unnamed: 0,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,proc_gyro:magnitude_spectrum:log_energy_band4,proc_gyro:magnitude_spectrum:spectral_entropy,proc_gyro:magnitude_autocorrelation:period,proc_gyro:magnitude_autocorrelation:normalized_ac,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,proc_gyro:3d:ro_xy,proc_gyro:3d:ro_xz,proc_gyro:3d:ro_yz,raw_magnet:magnitude_stats:mean,raw_magnet:magnitude_stats:std,raw_magnet:magnitude_stats:moment3,raw_magnet:magnitude_stats:moment4,raw_magnet:magnitude_stats:percentile25,raw_magnet:magnitude_stats:percentile50,raw_magnet:magnitude_stats:percentile75,raw_magnet:magnitude_stats:value_entropy,raw_magnet:magnitude_stats:time_entropy,raw_magnet:magnitude_spectrum:log_energy_band0,raw_magnet:magnitude_spectrum:log_energy_band1,raw_magnet:magnitude_spectrum:log_energy_band2,raw_magnet:magnitude_spectrum:log_energy_band3,raw_magnet:magnitude_spectrum:log_energy_band4,raw_magnet:magnitude_spectrum:spectral_entropy,raw_magnet:magnitude_autocorrelation:period,raw_magnet:magnitude_autocorrelation:normalized_ac,raw_magnet:3d:mean_x,raw_magnet:3d:mean_y,raw_magnet:3d:mean_z,raw_magnet:3d:std_x,raw_magnet:3d:std_y,raw_magnet:3d:std_z,raw_magnet:3d:ro_xy,raw_magnet:3d:ro_xz,raw_magnet:3d:ro_yz,raw_magnet:avr_cosine_similarity_lag_range0,raw_magnet:avr_cosine_similarity_lag_range1,raw_magnet:avr_cosine_similarity_lag_range2,raw_magnet:avr_cosine_similarity_lag_range3,raw_magnet:avr_cosine_similarity_lag_range4,watch_acceleration:magnitude_stats:mean,watch_acceleration:magnitude_stats:std,watch_acceleration:magnitude_stats:moment3,watch_acceleration:magnitude_stats:moment4,watch_acceleration:magnitude_stats:percentile25,watch_acceleration:magnitude_stats:percentile50,watch_acceleration:magnitude_stats:percentile75,watch_acceleration:magnitude_stats:value_entropy,watch_acceleration:magnitude_stats:time_entropy,watch_acceleration:magnitude_spectrum:log_energy_band0,watch_acceleration:magnitude_spectrum:log_energy_band1,watch_acceleration:magnitude_spectrum:log_energy_band2,watch_acceleration:magnitude_spectrum:log_energy_band3,watch_acceleration:magnitude_spectrum:log_energy_band4,watch_acceleration:magnitude_spectrum:spectral_entropy,...,audio_naive:mfcc10:std,audio_naive:mfcc11:std,audio_naive:mfcc12:std,audio_properties:max_abs_value,audio_properties:normalization_multiplier,discrete:app_state:is_active,discrete:app_state:is_inactive,discrete:app_state:is_background,discrete:app_state:missing,discrete:battery_plugged:is_ac,discrete:battery_plugged:is_usb,discrete:battery_plugged:is_wireless,discrete:battery_plugged:missing,discrete:battery_state:is_unknown,discrete:ba

In [12]:
master_df.describe()

,Unnamed: 0,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,raw_acc:magnitude_spectrum:log_energy_band1,raw_acc:magnitude_spectrum:log_energy_band2,raw_acc:magnitude_spectrum:log_energy_band3,raw_acc:magnitude_spectrum:log_energy_band4,raw_acc:magnitude_spectrum:spectral_entropy,raw_acc:magnitude_autocorrelation:period,raw_acc:magnitude_autocorrelation:normalized_ac,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,proc_gyro:magnitude_stats:mean,proc_gyro:magnitude_stats:std,proc_gyro:magnitude_stats:moment3,proc_gyro:magnitude_stats:moment4,proc_gyro:magnitude_stats:percentile25,proc_gyro:magnitude_stats:percentile50,proc_gyro:magnitude_stats:percentile75,proc_gyro:magnitude_stats:value_entropy,proc_gyro:magnitude_stats:time_entropy,proc_gyro:magnitude_spectrum:log_energy_band0,proc_gyro:magnitude_spectrum:log_energy_band1,proc_gyro:magnitude_spectrum:log_energy_band2,proc_gyro:magnitude_spectrum:log_energy_band3,proc_gyro:magnitude_spectrum:log_energy_band4,proc_gyro:magnitude_spectrum:spectral_entropy,proc_gyro:magnitude_autocorrelation:period,proc_gyro:magnitude_autocorrelation:normalized_ac,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,proc_gyro:3d:std_x,proc_gyro:3d:std_y,proc_gyro:3d:std_z,proc_gyro:3d:ro_xy,proc_gyro:3d:ro_xz,proc_gyro:3d:ro_yz,raw_magnet:magnitude_stats:mean,raw_magnet:magnitude_stats:std,raw_magnet:magnitude_stats:moment3,raw_magnet:magnitude_stats:moment4,raw_magnet:magnitude_stats:percentile25,raw_magnet:magnitude_stats:percentile50,raw_magnet:magnitude_stats:percentile75,raw_magnet:magnitude_stats:value_entropy,raw_magnet:magnitude_stats:time_entropy,raw_magnet:magnitude_spectrum:log_energy_band0,raw_magnet:magnitude_spectrum:log_energy_band1,raw_magnet:magnitude_spectrum:log_energy_band2,raw_magnet:magnitude_spectrum:log_energy_band3,raw_magnet:magnitude_spectrum:log_energy_band4,raw_magnet:magnitude_spectrum:spectral_entropy,raw_magnet:magnitude_autocorrelation:period,raw_magnet:magnitude_autocorrelation:normalized_ac,raw_magnet:3d:mean_x,raw_magnet:3d:mean_y,raw_magnet:3d:mean_z,raw_magnet:3d:std_x,raw_magnet:3d:std_y,raw_magnet:3d:std_z,raw_magnet:3d:ro_xy,raw_magnet:3d:ro_xz,raw_magnet:3d:ro_yz,raw_magnet:avr_cosine_similarity_lag_range0,raw_magnet:avr_cosine_similarity_lag_range1,raw_magnet:avr_cosine_similarity_lag_range2,raw_magnet:avr_cosine_similarity_lag_range3,raw_magnet:avr_cosine_similarity_lag_range4,watch_acceleration:magnitude_stats:mean,watch_acceleration:magnitude_stats:std,watch_acceleration:magnitude_stats:moment3,watch_acceleration:magnitude_stats:moment4,watch_acceleration:magnitude_stats:percentile25,watch_acceleration:magnitude_stats:percentile50,watch_acceleration:magnitude_stats:percentile75,watch_acceleration:magnitude_stats:value_entropy,watch_acceleration:magnitude_stats:time_entropy,watch_acceleration:magnitude_spectrum:log_energy_band0,watch_acceleration:magnitude_spectrum:log_energy_band1,watch_acceleration:magnitude_spectrum:log_energy_band2,watch_acceleration:magnitude_spectrum:log_energy_band3,watch_acceleration:magnitude_spectrum:log_energy_band4,watch_acceleration:magnitude_spectrum:spectral_entropy,...,audio_naive:mfcc10:std,audio_naive:mfcc11:std,audio_naive:mfcc12:std,audio_properties:max_abs_value,audio_properties:normalization_multiplier,discrete:app_state:is_active,discrete:app_state:is_inactive,discrete:app_state:is_background,discrete:app_state:missing,discrete:battery_plugged:is_ac,discrete:battery_plugged:is_usb,discrete:battery_plugged:is_wireless,discrete:battery_plugged:missing,discrete:battery_state:is_unknown,discrete:ba

## Data Cleaning

Most of data cleaning was done in the process of adding individual users to master dataframe. (commented out cell under imports)

In [13]:
master_nulls_list = list(master_df.loc[:,(master_df.isnull().sum()/master_df.isnull().count()*100 > 0).values].columns)

#some observations are still nan because interpolation doesn't work when all observations are nans. Use mean impuation for those features.
for col in master_nulls_list:
    master_df[col].fillna(master_df[col].mean(), inplace=True)

## Data Visualizations


### Target Variable 


In [14]:
label_col = label_col = [col for col in list(master_df.columns) if col[:6] == 'label:']

primary_labels = ['label:LYING_DOWN', 'label:SITTING', 'label:FIX_running', 'label:OR_standing','label:SLEEPING', 
               'label:FIX_walking']

loc_labels = ['label:LAB_WORK', 'label:IN_CLASS', 'label:IN_A_MEETING', 'label:LOC_main_workplace','label:OR_indoors',
 'label:OR_outside', 'label:IN_A_CAR', 'label:ON_A_BUS', 'label:LOC_home', 'label:FIX_restaurant','label:SHOPPING',
'label:AT_A_PARTY', 'label:AT_A_BAR', 'label:LOC_beach', 'label:AT_THE_GYM', 'label:ELEVATOR', 'label:AT_SCHOOL']

not_secondary_labels = primary_labels + loc_labels
secondary_labels = [col for col in label_col if col not in not_secondary_labels]

label_dict = {'primary_label': primary_labels, 'loc_label': loc_labels, 'secondary_label': secondary_labels}

for label, lst in label_dict.items():
    #init dict
    df_label_dict = {i: '' for i in range(len(master_df))}
    # add labels to dict if labels is present in respective index
    for i in range(len(master_df)):
        for col in lst:
            if master_df[col].iloc[i] == 1:
                #creates multiclass label
                df_label_dict[i] += col + ' '
    master_df[label] = pd.Series(df_label_dict).apply(lambda x: x if x != '' else np.NaN)

#plot label distribution for each category of labels 

NameError: name 'main_labels' is not defined

In [ ]:
len(secondary_labels)

In [ ]:
label_dict = {'primary_label': primary_labels, 'loc_label': loc_labels, 'secondary_label': secondary_labels}

for label in label_dict.keys():
    
    label_freq = master_df[label].value_counts()[:10]
    plt.figure(figsize=(25,5.5))
    plt.tick_params(labelsize=30)
    plt.barh(label_freq.index, label_freq.values)
    plt.title('{} Distribution'.format(label), fontdict={'fontsize': 30})
    
    plt.show()

    

In [31]:
len(label_col) - len(indoors_labels)
len(indoors_labels)

31

In [21]:
active_labels = ['label:FIX_walking', 'label:FIX_running', 'label:BICYCLING', 'label:OR_exercise', 'label:STROLLING',
                 'label:AT_THE_GYM', 'label:STAIRS_-_GOING_UP', 'label:STAIRS_-_GOING_DOWN']
with_people_labels = ['label:IN_A_MEETING', 'label:TALKING',  'label:WITH_CO-WORKERS', 'label:WITH_FRIENDS', 'label:LAB_WORK',
                       'label:IN_CLASS', 'label:DRIVE_-_I_M_A_PASSENGER', 'label:FIX_restaurant', 'label:AT_A_PARTY',
                      'label:AT_A_BAR','label:AT_SCHOOL']
indoors_labels = ['label:OR_indoors', 'label:SLEEPING', 'label:LAB_WORK', 'label:IN_CLASS', 'label:IN_A_MEETING',
                  'label:LOC_main_workplace', 'label:IN_A_CAR', 'label:ON_A_BUS', 'label:DRIVE_-_I_M_THE_DRIVER',
                  'label:DRIVE_-_I_M_A_PASSENGER', 'label:LOC_home', 'label:FIX_restaurant','label:SHOPPING',
                  'label:BATHING_-_SHOWER', 'label:CLEANING', 'label:DOING_LAUNDRY', 'label:WASHING_DISHES',
                  'label:WATCHING_TV', 'label:SURFING_THE_INTERNET', 'label:AT_A_PARTY','label:AT_A_BAR', 
                  'label:COMPUTER_WORK', 'label:EATING', 'label:TOILET', 'label:GROOMING', 'label:DRESSING',
                  'label:AT_THE_GYM', 'label:STAIRS_-_GOING_UP', 'label:STAIRS_-_GOING_DOWN','label:ELEVATOR',
                  'label:AT_SCHOOL']


In [16]:
def indoor_filter(df, indoors_labels):
    total_labels = 0
    for label in indoors_labels:
        if (df[label] == 1) & (df['label:OR_outside'] != 1):
            total_labels += 1
            break
            
    if total_labels >= 1:
        return 1
    else:
        return 0
            

In [17]:
master_df['label:INDOORS'] = 0
master_df['label:INDOORS'] = master_df.apply(lambda x: indoor_filter(x, indoors_labels), axis=1)
master_df['label:INDOORS'].value_counts()

1    285114
0    103762
Name: label:INDOORS, dtype: int64

In [ ]:
master_df['label:WITH_PEOPLE'] = 0
master_df['label:WITH_PEOPLE'] = np.where(master_df[with_people_labels] ==1, 1,0)
master_df['label:WITH_PEOPLE'].value_counts()

In [ ]:
master_df['label:ACTIVE'] = 0
master_df['label:ACTIVE'] = np.where(master_df[active_labels] ==1, 1,0)
master_df['label:ACTIVE'].value_counts()

In [ ]:
label_dict = {'label:ACTIVE': active_labels, 'label:INDOORS': indoors_labels, 'label:WITH_PEOPLE': with_people_labels}

for label in label_dict.keys():
    
    label_freq = master_df[label].value_counts()
    plt.figure(figsize=(25,3
                       ))
    plt.tick_params(labelsize=20)
    plt.barh(label_freq.index, label_freq.values)
    plt.title('{} Distribution'.format(label), fontdict={'fontsize': 30})
    
    plt.show()

    

__<font size="5">Research Question </font>__

Which classification model has the highest AUC score for each of the target behavioral labels: sitting, indoors and phone on table? 


In [ ]:
label_col = [col for col in list(master_df.columns) if col[:6] == 'label:']
all_feature_list = [col for col in list(master_df.columns) if col not in label_col and col != 'timestamp']


### Features

After looking at the coefficient correlations of all binary target variables, none of the sensor and self-reported features had a strong correlation (correlation > .7). Using a t-test between the true binary and false binary samples for each independent feature may be helpful in determining if there is a significant difference between the true and false binary samples. 

In [18]:
def t_test_plot_filter(data, feature_list, binary_label, pval_threshold=.05):
    
    t_test_dict = {}
    
    for i,col in enumerate(feature_list):
            zeros_series = data[col][data[binary_label] == 0]
            ones_series = data[col][data[binary_label] == 1]
            
            t_test, pval = ttest_ind(zeros_series, ones_series, equal_var=False)[0], ttest_ind(zeros_series, ones_series, equal_var=False)[1]
                
            if str(t_test) != 'nan' and pval < .05:
                t_test_dict[col] = abs(t_test)
    plot_feature_list = [col for col, t_val in t_test_dict.items() if t_val > np.array(list(t_test_dict.values())).mean()*2]
    
    return plot_feature_list

In [ ]:
#features that pass the t_test_plot_filter function are then used for violin plot visualizations
def target_violin_plot(data, feature_list, binary_label):
    for i,_ in enumerate(feature_list):
        
        plt.figure(figsize=(15,5))

        plt.subplot(1,2,1)
        sns.violinplot(x=binary_label, y=feature_list[i], data=data)
        
        
        try: 
            plt.subplot(1,2,2)
            sns.violinplot(x=binary_label, y=feature_list[i+1], data=data)
        except IndexError:
            break
        plt.tight_layout()
        plt.show()
        

In [19]:
indoors_feature_list = t_test_plot_filter(master_df, all_feature_list, 'label:INDOORS')
print('Feature Count: ', len(indoors_feature_list))
#target_violin_plot(master_df, indoor_feature_list[:10],'label:OR_indoors')

Feature Count:  24


In [ ]:
active_feature_list = t_test_plot_filter(master_df, all_feature_list, 'label:ACTIVE')
print('Feature Count: ', len(active_feature_list))
#target_violin_plot(master_df, sitting_feature_list[:10], 'label:PHONE_ON_TABLE')

In [ ]:
active_feature_list

In [ ]:
with_people_feature_list = t_test_plot_filter(master_df, all_feature_list, 'label:WITH_PEOPLE')
print('Feature Count: ', len(with_people_feature_list))
#target_violin_plot(master_df, phone_on_table_feature_list[:10], 'label:PHONE_ON_TABLE')

In [20]:
indoors_feature_list

['raw_acc:magnitude_stats:std',
 'raw_acc:magnitude_stats:moment3',
 'raw_acc:magnitude_stats:moment4',
 'raw_acc:magnitude_spectrum:log_energy_band2',
 'raw_acc:magnitude_spectrum:log_energy_band3',
 'raw_acc:magnitude_spectrum:log_energy_band4',
 'raw_acc:magnitude_spectrum:spectral_entropy',
 'raw_acc:3d:std_x',
 'raw_acc:3d:std_y',
 'raw_acc:3d:std_z',
 'proc_gyro:magnitude_stats:mean',
 'proc_gyro:magnitude_stats:std',
 'proc_gyro:magnitude_stats:percentile50',
 'proc_gyro:magnitude_stats:percentile75',
 'proc_gyro:3d:std_x',
 'proc_gyro:3d:std_y',
 'proc_gyro:3d:std_z',
 'watch_acceleration:magnitude_spectrum:log_energy_band2',
 'audio_naive:mfcc1:mean',
 'discrete:battery_state:is_full',
 'discrete:wifi_status:is_not_reachable',
 'discrete:wifi_status:is_reachable_via_wifi',
 'discrete:time_of_day:between0and6',
 'label_source']

# Modeling

Since there are observations within the master dataframe, we will downsample

In [87]:
#downsamples majority class label to be equal with minority class label
#used for models with large amounts of data to decrease runtime
def downsample_df(data, binary_label):
    drop_size = abs(len(data[data[binary_label] == 0]) - len(data[data[binary_label] ==1]))

    if len(data[data[binary_label] == 0]) > len(data[data[binary_label] ==1]):
        drop_idx = data[data[binary_label] == 0].sample(drop_size).index
    else:
        drop_idx = data[data[binary_label] == 1].sample(drop_size).index
        
    balanced_df = data.drop(list(drop_idx)).copy()
    print('Label Distribution')
    print(balanced_df[binary_label].value_counts())
    return balanced_df

In [99]:
def best_model_dict(model_dict, feature_list, binary_label, tiny_sample=True):
    if tiny_sample == True:
        data = downsample_df(master_df.sample(frac=.05), binary_label)
    else:
        data = downsample_df(master_df, binary_label)
    for name, model in model_dict.items():
        clf = GridSearchCV(model, model.parameters, cv=5, scoring='roc_auc')
        if name not in ['mlp', 'knn']:
            clf.fit(data[feature_list], data[binary_label])
        else:
            clf.fit(normalize(data[feature_list]), data[binary_label])
        
        model_dict[name] = clf.best_estimator_
    
    return model_dict

In [89]:
def test_best_model(data, model_dict, feature_list, binary_label, downsample=False):
    
    if downsample == True:
        data = downsample_df(data, binary_label)
    else: 
        print('Label Distribution')
        print(data[binary_label].value_counts())
    
    
    for name,model in model_dict.items():
        if name not in ['knn', 'mlp']:
            train_x, test_x, train_y, test_y = train_test_split(data[feature_list], data[binary_label], test_size=.3)
        else:
            #normalize features
            train_x, test_x, train_y, test_y = train_test_split(normalize(data[feature_list]), data[binary_label], test_size=.3)
        print()
        print(name)
        start = time.time()
        model.fit(train_x, train_y)
        end = time.time()
        score = model.score(test_x, test_y) * 100
        base = max(np.unique(test_y, return_counts=True)[1])/len(test_y) * 100
        pred_y = model.predict(test_x)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(test_y, pred_y)
        roc_auc = auc(false_positive_rate, true_positive_rate)*100
        
        print('Base: ', base)
        print('AUC: ', roc_auc)
        print('Runtime: ', end - start)
    



## Active Model

In [92]:
rfc = RandomForestClassifier()
rfc.parameters = {'criterion':('gini', 'entropy'), 'n_estimators':[5,10,20,40], 
                  'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

dt = DecisionTreeClassifier()
dt.parameters = {'criterion':('gini', 'entropy'), 'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

knn = KNeighborsClassifier()
knn.parameters = {'weights':('uniform', 'distance'), 'n_neighbors':[1,2,4,8]}

mlp = MLPClassifier()
mlp.parameters = {'hidden_layer_sizes':[10,15,20], 'activation':('relu', 'logistic'), 
              'learning_rate': ('constant', 'adaptive')}

active_model_dict = {'rfc': rfc, 'mlp':mlp, 'knn':knn, 'dt': dt}

active_model_dict = best_model_dict(active_model_dict, active_feature_list, 'label:ACTIVE')

active_model_dict

Label Distribution
1    1126
0    1126
Name: label:ACTIVE, dtype: int64
Label Distribution
1    1120
0    1120
Name: label:ACTIVE, dtype: int64


/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached an

/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached an

Label Distribution
1    1129
0    1129
Name: label:ACTIVE, dtype: int64
Label Distribution
1    1138
0    1138
Name: label:ACTIVE, dtype: int64


{'rfc': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=32, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=2, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'mlp': MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=20, learning_rate='constant',
        learning_rate_init=0.001, max_iter=200, momentum=0.9,
        n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
        random_state=None, shuffle=True, solver='adam', tol=0.0001,
        validation_fraction=0.1, verbose=False, warm_start=False),
 'knn': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            m

In [93]:
test_best_model(master_df, active_model_dict, active_feature_list, 'label:ACTIVE', downsample=True)

Label Distribution
1    22458
0    22458
Name: label:ACTIVE, dtype: int64

rfc
Base:  50.09276437847866
AUC:  83.2749277158199
Runtime:  8.40872597694397

mlp


/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Base:  50.14471243042672
AUC:  79.23919591529418
Runtime:  24.16994833946228

knn
Base:  50.17439703153988
AUC:  80.55433841702906
Runtime:  0.06574678421020508

dt
Base:  50.45640074211503
AUC:  77.64599299177945
Runtime:  0.5059311389923096


## Indoors 

In [96]:
rfc = RandomForestClassifier()
rfc.parameters = {'criterion':('gini', 'entropy'), 'n_estimators':[5,10,20,40], 
                  'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

dt = DecisionTreeClassifier()
dt.parameters = {'criterion':('gini', 'entropy'), 'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

knn = KNeighborsClassifier()
knn.parameters = {'weights':('uniform', 'distance'), 'n_neighbors':[1,2,4,8]}

mlp = MLPClassifier()
mlp.parameters = {'hidden_layer_sizes':[10,15,20], 'activation':('relu', 'logistic'), 
              'learning_rate': ('constant', 'adaptive')}

indoors_model_dict = {'rfc': rfc, 'mlp':mlp, 'knn':knn, 'dt': dt}

indoors_model_dict = best_model_dict(indoors_model_dict, indoors_feature_list, 'label:INDOORS')

indoors_model_dict

Label Distribution
1    5160
0    5160
Name: label:INDOORS, dtype: int64
Label Distribution
1    5324
0    5324
Name: label:INDOORS, dtype: int64


/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached an

Label Distribution
1    5262
0    5262
Name: label:INDOORS, dtype: int64
Label Distribution
1    5160
0    5160
Name: label:INDOORS, dtype: int64


{'rfc': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=16, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'mlp': MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=20, learning_rate='constant',
        learning_rate_init=0.001, max_iter=200, momentum=0.9,
        n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
        random_state=None, shuffle=True, solver='adam', tol=0.0001,
        validation_fraction=0.1, verbose=False, warm_start=False),
 'knn': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            m

In [97]:
test_best_model(master_df, indoors_model_dict, indoors_feature_list, 'label:INDOORS', downsample=True)

Label Distribution
1    103762
0    103762
Name: label:INDOORS, dtype: int64

rfc
Base:  50.12528510392239
AUC:  84.6219131994724
Runtime:  25.892632246017456

mlp
Base:  50.1943525330078
AUC:  78.53401062951497
Runtime:  73.99514508247375

knn
Base:  50.26502618137428
AUC:  80.91184065490485
Runtime:  0.5505168437957764

dt
Base:  50.10601047254971
AUC:  80.61177204120182
Runtime:  3.2313342094421387


## With Friends


In [100]:
rfc = RandomForestClassifier()
rfc.parameters = {'criterion':('gini', 'entropy'), 'n_estimators':[5,10,20,40], 
                  'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

dt = DecisionTreeClassifier()
dt.parameters = {'criterion':('gini', 'entropy'), 'max_depth': [1,2,4,8,16,32], 'min_samples_leaf': [1,2,4,8]}

knn = KNeighborsClassifier()
knn.parameters = {'weights':('uniform', 'distance'), 'n_neighbors':[1,2,4,8]}

mlp = MLPClassifier()
mlp.parameters = {'hidden_layer_sizes':[10,15,20], 'activation':('relu', 'logistic'), 
              'learning_rate': ('constant', 'adaptive')}

with_people_model_dict = {'rfc': rfc, 'mlp':mlp, 'knn':knn, 'dt': dt}

with_people_model_dict_model_dict = best_model_dict(with_people_model_dict, with_people_feature_list, 'label:WITH_PEOPLE', tiny_sample=False)

with_people_model_dict

Label Distribution
1    5504
0    5504
Name: label:WITH_PEOPLE, dtype: int64


/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached an

/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached an

{'rfc': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=32, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=4, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'mlp': MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=20, learning_rate='constant',
        learning_rate_init=0.001, max_iter=200, momentum=0.9,
        n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
        random_state=None, shuffle=True, solver='adam', tol=0.0001,
        validation_fraction=0.1, verbose=False, warm_start=False),
 'knn': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            m

In [101]:
test_best_model(master_df, with_people_model_dict, with_people_feature_list, 'label:WITH_PEOPLE', downsample=True)


Label Distribution
1    5504
0    5504
Name: label:WITH_PEOPLE, dtype: int64

rfc
Base:  50.46927036027854
AUC:  89.58373973127135
Runtime:  1.0962400436401367

mlp


/Users/marshallmamiya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Base:  50.62064789585226
AUC:  84.60789972394896
Runtime:  3.407088041305542

knn
Base:  50.196790796245836
AUC:  87.40317737340554
Runtime:  0.008879661560058594

dt
Base:  50.13623978201635
AUC:  81.04211214065099
Runtime:  0.05835223197937012


# Evaluate Best Models: RFC

Best Models:
    - Active Label: RFC
    - Indoors Label: RFC
    - With People Label: RFC

In [102]:
cross_val_dict = {'label:ACTIVE': ['rfc', active_model_dict['rfc'], active_feature_list], 
                  'label:INDOORS': ['rfc', indoors_model_dict['rfc'], indoors_feature_list], 
                  'label:WITH_PEOPLE': ['rfc', with_people_model_dict['rfc'], with_people_feature_list]}


In [103]:
for label, model in cross_val_dict.items():
    
    data = master_df[model[2]]
        
    scores = cross_val_score(model[1], data, master_df[label], scoring='roc_auc', cv=5)
    print()
    print(model[0])
    print(label)
    print("AUC: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


rfc
label:ACTIVE
AUC: 0.83 (+/- 0.07)

rfc
label:INDOORS
AUC: 0.75 (+/- 0.15)

rfc
label:WITH_PEOPLE
AUC: 0.84 (+/- 0.09)
